In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from sklearn.metrics import mean_absolute_error
import os
import optuna
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.linear_model import Ridge, Lasso, ElasticNet

import csv
import os
import json
import numpy as np
from datetime import datetime

In [ ]:

folder = '../data/processed'
filename = 'dataset_final.csv'
file_path = os.path.join(folder, filename)

try:
    df = pd.read_csv(
        file_path,
        sep=',',
        decimal='.'
    )
    
    print(f"✅ DataFrame cargado exitosamente desde: {file_path}")
    print(f"Dimensiones: {df.shape}")
    print("\nPrimeras 5 filas:")
    print(df.head())

except FileNotFoundError:
    print(f"❌ Error: No se encontró el archivo en la ruta: {file_path}")
except Exception as e:
    print(f"❌ Error al leer el archivo: {e}")

In [ ]:

# 1️⃣ Cargar dataset definitivo
ruta_csv = "../data/processed/X_test_preproc.csv"
X_test = pd.read_csv(ruta_csv, sep=',', decimal='.')
ruta_csv = "../data/processed/X_train_preproc.csv"
X_train = pd.read_csv(ruta_csv, sep=',', decimal='.')
ruta_csv = "../data/processed/y_test.csv"
y_test = pd.read_csv(ruta_csv, sep=',', decimal='.')
ruta_csv = "../data/processed/y_train.csv"
y_train = pd.read_csv(ruta_csv, sep=',', decimal='.')


print("Shape dataset:", df.shape)




In [ ]:
def objective_rf_mae(trial):
    # Hiperparámetros
    n_estimators = trial.suggest_int("n_estimators", 100, 1000, step=50)
    max_depth = trial.suggest_int("max_depth", 5, 50)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 20)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)
    max_features = trial.suggest_float("max_features", 0.1, 1.0, log=True)

    # Modelo
    model = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        n_jobs=-1,
        random_state=42
    )
    
    # CV y Métrica (MAE)
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring="neg_mean_absolute_error", n_jobs=-1)
    
    return -np.mean(scores)

In [ ]:
def objective_xgb_mae(trial):
    # Hiperparámetros
    n_estimators = trial.suggest_int("n_estimators", 100, 2000, step=100)
    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.3, log=True)
    max_depth = trial.suggest_int("max_depth", 3, 10)
    subsample = trial.suggest_float("subsample", 0.6, 1.0, step=0.1)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.6, 1.0, step=0.1)
    reg_alpha = trial.suggest_float("reg_alpha", 1e-8, 1.0, log=True) # L1
    reg_lambda = trial.suggest_float("reg_lambda", 1e-8, 1.0, log=True) # L2

    # Modelo
    model = xgb.XGBRegressor(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        n_jobs=-1,
        random_state=42,
        verbosity=0
    )
    
    # CV y Métrica (MAE)
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring="neg_mean_absolute_error", n_jobs=-1)
    
    return -np.mean(scores)

In [ ]:
def objective_lgbm_mae(trial):
    # Hiperparámetros
    n_estimators = trial.suggest_int("n_estimators", 100, 2000, step=100)
    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.3, log=True)
    num_leaves = trial.suggest_int("num_leaves", 20, 150)
    subsample = trial.suggest_float("subsample", 0.6, 1.0, step=0.1)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.6, 1.0, step=0.1)
    reg_alpha = trial.suggest_float("reg_alpha", 1e-8, 1.0, log=True) # L1
    reg_lambda = trial.suggest_float("reg_lambda", 1e-8, 1.0, log=True) # L2

    # Modelo
    model = lgb.LGBMRegressor(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        num_leaves=num_leaves,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        n_jobs=-1,
        random_state=42,
        verbose=-1
    )
    
    # CV y Métrica (MAE)
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring="neg_mean_absolute_error", n_jobs=-1)
    
    return -np.mean(scores)

In [ ]:
def objective_elasticnet_mae(trial):
    alpha = trial.suggest_float("alpha", 1e-3, 10.0, log=True)
    l1_ratio = trial.suggest_float("l1_ratio", 0.0, 1.0) 

    model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42, max_iter=2000)
    
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring="neg_mean_absolute_error", n_jobs=-1)
    
    return -np.mean(scores)

# 1. Crear los estudios (todos buscan minimizar el MAE)
study_rf_mae = optuna.create_study(direction="minimize")
study_xgb_mae = optuna.create_study(direction="minimize")
study_lgbm_mae = optuna.create_study(direction="minimize")
study_elasticnet_mae = optuna.create_study(direction="minimize")

# 2. Ejecutar las optimizaciones
print("Optimizando Random Forest (MAE)...")
study_rf_mae.optimize(objective_rf_mae, n_trials=50)

print("Optimizando XGBoost (MAE)...")
study_xgb_mae.optimize(objective_xgb_mae, n_trials=75)

print("Optimizando LightGBM (MAE)...")
study_lgbm_mae.optimize(objective_lgbm_mae, n_trials=75)

print("Optimizando ElasticNet (MAE)...")
study_elasticnet_mae.optimize(objective_elasticnet_mae, n_trials=40)

# 3. Ver los mejores parámetros
print(f"Mejor RF (MAE): {study_rf_mae.best_value} con params {study_rf_mae.best_params}")
print(f"Mejor XGB (MAE): {study_xgb_mae.best_value} con params {study_xgb_mae.best_params}")
print(f"Mejor LGBM (MAE): {study_lgbm_mae.best_value} con params {study_lgbm_mae.best_params}")
print(f"Mejor ElasticNet (MAE): {study_elasticnet_mae.best_value} con params {study_elasticnet_mae.best_params}")

In [ ]:

def log_experiment(model_name: str, 
                   model_object: object, 
                   study: optuna.study.Study, 
                   X_train, y_train, X_test, y_test):
    """
    Entrena un modelo con los mejores parámetros de un estudio de Optuna,
    calcula métricas y las guarda en un CSV.
    """
    
    # --- 1. Definir la ruta del log ---
    log_dir = "results"
    log_file = os.path.join(log_dir, "experiment_logs.csv")
    os.makedirs(log_dir, exist_ok=True) # Crea la carpeta si no existe

    # --- 2. Obtener mejores parámetros y valor del estudio ---
    best_params = study.best_params
    best_value = study.best_value # El MAE de CV que encontró Optuna
    
    # --- 3. Instanciar y entrenar el modelo final ---
    # Manejo especial para modelos lineales que necesitan Pipeline
    if "pipeline" in model_name.lower():
        # El pipeline ya está definido en la función objective, 
        # aquí solo re-creamos el mejor.
        if model_name == "Pipeline_ElasticNet":
             # Re-crea el pipeline con los mejores params
             model =ElasticNet(
                    alpha=best_params.get('alpha'), 
                    l1_ratio=best_params.get('l1_ratio'),
                    random_state=42, 
                    max_iter=2000
                )
            
            
    else:
        # Para modelos no-pipeline (RF, XGB, LGBM)
        model = model_object(**best_params, random_state=42, n_jobs=-1)
        # Ajustes para modelos que no les gusta 'n_jobs' o necesitan 'verbose'
        if isinstance(model, (xgb.XGBRegressor, lgb.LGBMRegressor)):
            model.set_params(verbosity=0) if isinstance(model, xgb.XGBRegressor) else None
            model.set_params(verbose=-1) if isinstance(model, lgb.LGBMRegressor) else None

    # Entrenar el modelo final con todos los datos de train
    model.fit(X_train, y_train)

    # --- 4. Calcular Métricas (Train y Test) ---
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    metrics = {
        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'model_name': model_name,
        'optuna_cv_mae': best_value, # Métrica optimizada
        'mae_train': mean_absolute_error(y_train, y_pred_train),
        'mae_test': mean_absolute_error(y_test, y_pred_test),
        'rmse_train': np.sqrt(mean_squared_error(y_train, y_pred_train)),
        'rmse_test': np.sqrt(mean_squared_error(y_test, y_pred_test)),
        'r2_train': r2_score(y_train, y_pred_train),
        'r2_test': r2_score(y_test, y_pred_test),
        'hyperparameters': json.dumps(best_params) # Guardar params como string JSON
    }
    
    # --- 5. Escribir en el CSV ---
    file_exists = os.path.isfile(log_file)
    
    with open(log_file, 'a', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=metrics.keys())
        
        if not file_exists:
            writer.writeheader() # Escribir cabecera solo si el archivo es nuevo
            
        writer.writerow(metrics)
        
    print(f"Resultados de '{model_name}' guardados en {log_file}")

In [ ]:
# --- 1. Ejecutar el estudio (ejemplo con Random Forest) ---
print("Optimizando Random Forest (MAE)...")
study_rf_mae = optuna.create_study(direction="minimize")
# (Aquí asumimos que 'objective_rf_mae' está definida)
study_rf_mae.optimize(objective_rf_mae, n_trials=50) 

# --- 2. Registrar el resultado ---
log_experiment(
    model_name="RandomForest",
    model_object=RandomForestRegressor, # Pasa la clase del modelo
    study=study_rf_mae,
    X_train=X_train, y_train=y_train,
    X_test=X_test, y_test=y_test
)

# --- 3. Ejecutar el estudio (ejemplo con XGBoost) ---
print("Optimizando XGBoost (MAE)...")
study_xgb_mae = optuna.create_study(direction="minimize")
# (Aquí asumimos que 'objective_xgb_mae' está definida)
study_xgb_mae.optimize(objective_xgb_mae, n_trials=50)

# --- 4. Registrar el resultado ---
log_experiment(
    model_name="XGBoost",
    model_object=xgb.XGBRegressor, # Pasa la clase del modelo
    study=study_xgb_mae,
    X_train=X_train, y_train=y_train,
    X_test=X_test, y_test=y_test
)

# --- 5. Ejecutar el estudio (ejemplo con ElasticNet) ---
print("Optimizando ElasticNet (MAE)...")
study_elasticnet_mae = optuna.create_study(direction="minimize")
# (Aquí asumimos que 'objective_elasticnet_mae' está definida)
study_elasticnet_mae.optimize(objective_elasticnet_mae, n_trials=40)

# --- 6. Registrar el resultado ---
# Nota: 'model_object' es None porque se maneja dentro del 'if'
log_experiment(
    model_name="Pipeline_ElasticNet",
    model_object=None, 
    study=study_elasticnet_mae,
    X_train=X_train, y_train=y_train,
    X_test=X_test, y_test=y_test
)

# Repetir para LightGBM...